# VacationPy
----

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import data csv file
weather_cities_data = os.path.join("output_data", "weather_data_cities.csv")

# Read weather data file and store into Pandas data frame
weather_data = pd.read_csv(weather_cities_data, encoding="utf-8")

weather_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.1203,-134.9692,79.21,75,7,10.27,PF,1616521508
1,1,George Town,5.4112,100.3354,78.80,94,20,1.14,MY,1616521508
2,2,Bluff,-46.6000,168.3333,53.01,98,95,4.70,NZ,1616521509
3,3,Dikson,73.5069,80.5464,-21.71,99,50,6.85,RU,1616521509
4,4,Waingapu,-9.6567,120.2641,78.19,84,95,5.03,ID,1616521509


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
weather_data_locs = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"]

import gmaps
gmaps.configure(api_key=g_key)
fig = gmaps.figure(center=(30, -5), zoom_level=2)
heat_layer = gmaps.heatmap_layer(weather_data_locs, weights=humidity, dissipating=False, 
                                 max_intensity=max(weather_data["Humidity"]), point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
vacation_cities = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & 
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)]

vacation_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,69,Hermanus,-34.4187,19.2345,70.00,74,0,8.01,ZA,1616521526
105,105,Marsh Harbour,26.5412,-77.0636,72.28,52,0,5.32,BS,1616521536
171,171,Pisco,-13.7000,-76.2167,75.20,64,0,9.22,PE,1616521553
212,212,Khanpur,28.6453,70.6567,75.38,19,0,5.95,PK,1616521564
304,304,Jaisalmer,26.9147,70.9181,79.92,16,0,4.23,IN,1616521588
337,337,Olavarría,-36.8927,-60.3225,75.99,51,0,5.01,AR,1616521597
401,401,Muscat,23.6139,58.5922,78.80,50,0,3.44,OM,1616521614
475,475,Najrān,17.4924,44.1277,78.80,13,0,4.61,SA,1616521635
534,534,Alice Town,25.7167,-79.3000,70.25,58,0,5.70,BS,1616521651


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels within 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = vacation_cities.reset_index(drop=True).set_index("City_ID")
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
City_ID,,,,,,,,,,,
69,Hermanus,-34.4187,19.2345,70.00,74,0,8.01,ZA,1616521526,,
105,Marsh Harbour,26.5412,-77.0636,72.28,52,0,5.32,BS,1616521536,,
171,Pisco,-13.7000,-76.2167,75.20,64,0,9.22,PE,1616521553,,
212,Khanpur,28.6453,70.6567,75.38,19,0,5.95,PK,1616521564,,
304,Jaisalmer,26.9147,70.9181,79.92,16,0,4.23,IN,1616521588,,


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():

    params = {
    "location": f"{row['Lat']}, {row['Lng']}", 
    "type": "hotel", 
    "keyword": "hotel", 
    "radius": 5000,
    "key": g_key
    }

    # get city location
    city = row['City']

    # assemble url and make API request
    print(f"Retrieving Results for: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel to {city} is {results[0]['name']}.")
        print(f"Address is {results[0]['vicinity']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Hotel Address'] = results[0]['vicinity']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for: Hermanus.
Closest hotel to Hermanus is Birkenhead House.
Address is 119 11th St, Hermanus.
------------
Retrieving Results for: Marsh Harbour.
Closest hotel to Marsh Harbour is Abaco Beach Resort.
Address is Bay St, Marsh Harbour.
------------
Retrieving Results for: Pisco.
Closest hotel to Pisco is Hostal Tambo Colorado.
Address is Francisco Bolognesi 159, Pisco.
------------
Retrieving Results for: Khanpur.
Closest hotel to Khanpur is Marhaba Guest House and Hotel.
Address is Doaba Colony Khanpur.
------------
Retrieving Results for: Jaisalmer.
Closest hotel to Jaisalmer is Shahi Palace Hotel.
Address is Shiv Street, Fort Rd, behind HDFC Bank, Jaisalmer.
------------
Retrieving Results for: Olavarría.
Closest hotel to Olavarría is La Josefina Hotel Boutique.
Address is Alsina 3045, Olavarría.
------------
Retrieving Results for: Muscat.
Closest hotel to Muscat is Al Bustan Palace, A Ritz-Carlton Hotel.
Address is OM, Al Bustan St.
------------
Retrieving Resul

In [7]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
City_ID,,,,,,,,,,,
69,Hermanus,-34.4187,19.2345,70.00,74,0,8.01,ZA,1616521526,Birkenhead House,"119 11th St, Hermanus"
105,Marsh Harbour,26.5412,-77.0636,72.28,52,0,5.32,BS,1616521536,Abaco Beach Resort,"Bay St, Marsh Harbour"
171,Pisco,-13.7000,-76.2167,75.20,64,0,9.22,PE,1616521553,Hostal Tambo Colorado,"Francisco Bolognesi 159, Pisco"
212,Khanpur,28.6453,70.6567,75.38,19,0,5.95,PK,1616521564,Marhaba Guest House and Hotel,Doaba Colony Khanpur
304,Jaisalmer,26.9147,70.9181,79.92,16,0,4.23,IN,1616521588,Shahi Palace Hotel,"Shiv Street, Fort Rd, behind HDFC Bank, Jaisalmer"
337,Olavarría,-36.8927,-60.3225,75.99,51,0,5.01,AR,1616521597,La Josefina Hotel Boutique,"Alsina 3045, Olavarría"
401,Muscat,23.6139,58.5922,78.80,50,0,3.44,OM,1616521614,"Al Bustan Palace, A Ritz-Carlton Hotel","OM, Al Bustan St"
475,Najrān,17.4924,44.1277,78.80,13,0,4.61,SA,1616521635,,
534,Alice Town,25.7167,-79.3000,70.25,58,0,5.70,BS,1616521651,Hilton at Resorts World Bimini,Hilton at Resorts World Bimini North


In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Address</dt><dd>{Hotel Address}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))